In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models

# Data loading and preprocessing
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'T:\pro\dinopedia\dinno'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
               for x in ['train', 'val']}

# Load AlexNet model
model = models.alexnet(pretrained=True)
model.classifier[6] = nn.Linear(model.classifier[6].in_features, 43)  # num of classes 43

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Reduced learning rate

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    print(f"Epoch {epoch}/{num_epochs}")
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in dataloaders[phase]:
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])
        print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

# Save model
torch.save(model.state_dict(), 'alexnet_dino_model.pth')

c:\Users\tarun\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tarun\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 0/10
train Loss: 2.5698 Acc: 0.3304
val Loss: 1.4348 Acc: 0.6352
Epoch 1/10
train Loss: 1.2963 Acc: 0.6324
val Loss: 0.7751 Acc: 0.8018
Epoch 2/10
train Loss: 0.7596 Acc: 0.7730
val Loss: 0.4957 Acc: 0.8841
Epoch 3/10
train Loss: 0.4910 Acc: 0.8529
val Loss: 0.3766 Acc: 0.9177
Epoch 4/10
train Loss: 0.3336 Acc: 0.9025
val Loss: 0.3308 Acc: 0.9284
Epoch 5/10
train Loss: 0.2367 Acc: 0.9290
val Loss: 0.2964 Acc: 0.9338
Epoch 6/10
train Loss: 0.1719 Acc: 0.9478
val Loss: 0.3172 Acc: 0.9342
Epoch 7/10
train Loss: 0.1630 Acc: 0.9496
val Loss: 0.3019 Acc: 0.9396
Epoch 8/10
train Loss: 0.1432 Acc: 0.9618
val Loss: 0.2752 Acc: 0.9435
Epoch 9/10
train Loss: 0.1180 Acc: 0.9665
val Loss: 0.2816 Acc: 0.9391


In [1]:
import wikipediaapi

def get_dinosaur_info(species_name):
    wiki_wiki = wikipediaapi.Wikipedia('en')
    page = wiki_wiki.page(species_name)

    if page.exists():
        info = {
            "species_name": species_name,
            "description": page.summary[:500],  # First 500 characters for brevity
            "url": page.fullurl
        }
        return info
    else:
        return {"error": "Dinosaur information not found"}


In [2]:
from transformers import pipeline

# Initialize the Hugging Face pipeline for text generation with GPU
generator = pipeline("text-generation", model="gpt2", device=0)

def generate_fun_fact(description):
    prompt = f"Tell me a fun fact about this dinosaur: {description}"
    fun_fact = generator(prompt, max_length=50, num_return_sequences=1, clean_up_tokenization_spaces=True)[0]["generated_text"]
    return fun_fact

c:\Users\tarun\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
import torch
import torchvision.transforms as transforms
from torchvision import models
from PIL import Image

# Define the AlexNet model architecture
model = models.alexnet(pretrained=False)

# Modify the classifier layer to match the number of classes in your trained model
num_classes = 43
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, num_classes)

# Load the state dictionary
model_path = "T:/pro/dinopedia/alexnet_dino_model1.pth"
model.load_state_dict(torch.load(model_path))
model.eval()

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def classify_dinosaur(image_file):
    # Open the image file
    image = Image.open(image_file)
    # Apply the transformations
    image = transform(image).unsqueeze(0)
    # Perform the classification
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    # Map the predicted index to the species name
    species_name = map_index_to_species(predicted.item())
    return species_name

def map_index_to_species(index):
    # This function should map the index to the actual species name
    # You need to implement this based on your model's training data
    species_mapping = {
    0: "Allosaurus",
    1: "Ankylosaurus",
    2: "Apatosaurus",
    3: "Baryonyx",
    4: "Brachiosaurus",
    5: "Carnotaurus",
    6: "Ceratosaurus",
    7: "Compsognathus",
    8: "Corythosaurus",
    9: "Dilophosaurus",
    10: "Dimetrodon",
    11: "Dimorphodon",
    12: "Dreadnoughtus",
    13: "Gallimimus",
    14: "Giganotosaurus",
    15: "Iguanodon",
    16: "Kentrosaurus",
    17: "Lystrosaurus",
    18: "Mamenchisaurus",
    19: "Microceratus",
    20: "Monolophosaurus",
    21: "Mosasaurus",
    22: "Nasutoceratops",
    23: "Nothosaurus",
    24: "Ouranosaurus",
    25: "Oviraptor",
    26: "Pachycephalosaurus",
    27: "Pachyrhinosaurus",
    28: "Parasaurolophus",
    29: "Pteranodon",
    30: "Pyroraptor",
    31: "Quetzalcoatlus",
    32: "Sinoceratops",
    33: "Smilodon",
    34: "Spinosaurus",
    35: "Stegosaurus",
    36: "Stygimoloch",
    37: "Suchomimus",
    38: "Tarbosaurus",
    39: "Therizinosaurus",
    40: "Triceratops",
    41: "Tyrannosaurus rex",
    42: "Velociraptor"
}
    return species_mapping.get(index, "Unknown species")

c:\Users\tarun\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tarun\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# streamlit run T:/pro/dinopedia/dinopedia_app.py

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

def download_flant5_model():
    model_name = 'google/flan-t5-large'
    cache_dir = 'T:/pro/dinopedia/flan-t5-large'  # Replace with your desired local path

    # Download the model and tokenizer
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name, cache_dir=cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
    
    
    return model, tokenizer

# Download the model and tokenizer
flan_model, flan_tokenizer = download_flant5_model()

c:\Users\tarun\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
